In [45]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [46]:
# 1. Obtener un set de datos.
df = pd.read_csv("data.csv")

# 2. Buscar valores vacios
print("Missing values:")
print(df.isna().sum())

# 3. Eliminar columna que no agrega valor al entrenamiento
df = df.drop('Employee_ID', axis=1)

Missing values:
Employee_ID                      0
Age                              0
Gender                           0
Marital_Status                   0
Department                       0
Job_Role                         0
Job_Level                        0
Monthly_Income                   0
Hourly_Rate                      0
Years_at_Company                 0
Years_in_Current_Role            0
Years_Since_Last_Promotion       0
Work_Life_Balance                0
Job_Satisfaction                 0
Performance_Rating               0
Training_Hours_Last_Year         0
Overtime                         0
Project_Count                    0
Average_Hours_Worked_Per_Week    0
Absenteeism                      0
Work_Environment_Satisfaction    0
Relationship_with_Manager        0
Job_Involvement                  0
Distance_From_Home               0
Number_of_Companies_Worked       0
Attrition                        0
dtype: int64


In [47]:
# Guardar las columnas categóricas
categorical_cols = ['Gender', 'Marital_Status', 'Department', 'Job_Role', 'Overtime']

# Guardar las columnas numéricas
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['Attrition']]

In [48]:
# 4. Separar features y target
X = df.drop('Attrition', axis=1)
y = df['Attrition']

In [49]:
# 5. Dividir en train y test con un split 80 - 20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
# 6. Aplicar las técnicas de escalamiento y transformación de datos
# Transformar datos categóricos mediante One Hot Encoder
encoder = OneHotEncoder(sparse_output=False)
X_train_cat_encoded = encoder.fit_transform(X_train[categorical_cols])
X_test_cat_encoded = encoder.transform(X_test[categorical_cols])

# Escalamiento de datos numéricos mediante StandarScaler
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[numerical_cols])
X_test_num_scaled = scaler.transform(X_test[numerical_cols])

In [51]:
# 7. Obtener nombres de features
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

# Crear DataFrames para datos categóricos transformados
X_train_cat_df = pd.DataFrame(X_train_cat_encoded, columns=encoded_feature_names)
X_test_cat_df = pd.DataFrame(X_test_cat_encoded, columns=encoded_feature_names)

# Crear DataFrames para datos numéricos transformados
X_train_num_df = pd.DataFrame(X_train_num_scaled, columns=numerical_cols)
X_test_num_df = pd.DataFrame(X_test_num_scaled, columns=numerical_cols)

In [52]:
# 8. Combinar datos categóricos y numéricos
X_train_processed = pd.concat([X_train_num_df, X_train_cat_df], axis=1)
X_test_processed = pd.concat([X_test_num_df, X_test_cat_df], axis=1)

In [53]:
# 9. Transformar target a valor numerico mediante LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [54]:
# 8. Implementación de Modelo 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

input_shape = X_train_processed.shape[1:]

model = Sequential([
    Flatten(input_shape=input_shape),  
    Dense(256, activation="relu"),  
    Dense(1, activation="sigmoid")   
])

/Users/sebastianflores/myenv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [55]:
# 9. Compilamos modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [56]:
# 10. Entrenamiento de modelo
history = model.fit(
    X_train_processed, y_train_encoded,
    epochs=50,
    validation_data=(X_test_processed, y_test_encoded)
)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 862us/step - accuracy: 0.7890 - loss: 0.5189 - val_accuracy: 0.7960 - val_loss: 0.5123
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - accuracy: 0.7992 - loss: 0.4993 - val_accuracy: 0.7960 - val_loss: 0.5236
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - accuracy: 0.7967 - loss: 0.4967 - val_accuracy: 0.7960 - val_loss: 0.5169
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - accuracy: 0.8112 - loss: 0.4687 - val_accuracy: 0.7960 - val_loss: 0.5205
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.7977 - loss: 0.4840 - val_accuracy: 0.7960 - val_loss: 0.5240
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - accuracy: 0.8072 - loss: 0.4648 - val_accuracy: 0.7960 - val_loss: 0.5255
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - accuracy: 0.8075 - loss: 0.4621 - val_accuracy: 0.7955 - val_loss: 0.5268
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - accuracy: 0.8060 - loss: 0.4595 - 

In [57]:
# 11. Evaluación inicial del modelo  
test_loss, test_acc = model.evaluate(X_test_processed, y_test_encoded)
print(f'Accuracy: {test_acc:.4f}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step - accuracy: 0.7369 - loss: 0.6734
Accuracy: 0.7335
